In [1]:
import numpy as np
import matplotlib as plt
import trimesh

In [2]:
class Canvas:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.pixels = np.zeros((height, width, 3), dtype=np.float32)  # RGB values

    def set_pixel(self, x, y, color):
        self.pixels[y, x] = np.clip(color, 0, 1)  # Ensuring the color stays within [0, 1]

    def save(self, filename):
        # Save the image using an external library, such as imageio
        import imageio
        imageio.imwrite(filename, (self.pixels * 255).astype(np.uint8))

In [3]:
class Ray:
    def __init__(self, origin, direction):
        self.origin = np.array(origin)
        self.direction = np.array(direction)

    def point_at_parameter(self, t):
        return self.origin + t * self.direction

In [4]:
class Camera:
    def __init__(self, position, look_at, up, fov, aspect_ratio):
        self.position = np.array(position)
        self.look_at = np.array(look_at)
        self.up = np.array(up)
        self.fov = fov
        self.aspect_ratio = aspect_ratio
        self._calculate_basis_vectors()

    def _calculate_basis_vectors(self):
        w = self.position - self.look_at
        self.w = w / np.linalg.norm(w)
        u = np.cross(self.up, self.w)
        self.u = u / np.linalg.norm(u)
        self.v = np.cross(self.w, self.u)

    def get_ray(self, u, v):
        # Generate ray direction in camera space and transform to world space
        alpha = np.tan(self.fov / 2) * ((2 * u - 1) * self.aspect_ratio)
        beta = np.tan(self.fov / 2) * (2 * v - 1)
        direction = alpha * self.u + beta * self.v - self.w
        return Ray(self.position, direction / np.linalg.norm(direction))

In [21]:
class Material:
    def __init__(self, color):
        self.color = np.array(color)

class Light:
    def __init__(self, position, intensity):
        self.position = np.array(position)
        self.intensity = intensity

In [22]:
class Triangle:
    def __init__(self, v0, v1, v2, material):
        self.v0 = np.array(v0)
        self.v1 = np.array(v1)
        self.v2 = np.array(v2)
        self.edge1 = self.v1 - self.v0
        self.edge2 = self.v2 - self.v0
        self.normal = np.cross(self.edge1, self.edge2)
        self.normal /= np.linalg.norm(self.normal)  # Normalisation du vecteur normal
        self.material = material

    def intersect(self, ray):
        epsilon = 1e-6
        h = np.cross(ray.direction, self.edge2)
        a = np.dot(self.edge1, h)
        
        if abs(a) < epsilon:
            return None  # Le rayon est parallèle à ce triangle.

        f = 1.0 / a
        s = ray.origin - self.v0
        u = f * np.dot(s, h)

        if not (0.0 <= u <= 1.0):
            return None

        q = np.cross(s, self.edge1)
        v = f * np.dot(ray.direction, q)

        if not (0.0 <= v <= 1.0 - u):
            return None

        t = f * np.dot(self.edge2, q)
        if t > epsilon:  # Intersection avec le rayon
            hit_point = ray.origin + ray.direction * t
            return {'distance': t, 'point': hit_point, 'normal': self.normal}
        else:
            return None  # Intersection avec la ligne mais pas avec le rayon.

def load_obj(filename):
    vertices = []
    triangles = []
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    for line in lines:
        if line.startswith('v '):
            vertices.append(list(map(float, line.strip().split()[1:])))
    
    material = Material([1, 1, 1])  # Assuming a simple material instance
    for line in lines:
        if line.startswith('f '):
            indices = [int(i.split('/')[0]) - 1 for i in line.strip().split()[1:]]
            triangles.append(Triangle(vertices[indices[0]], vertices[indices[1]], vertices[indices[2]], material))

    return triangles

In [23]:
def trace_ray(ray, objects, lights, depth=1):
    if depth == 0:
        return np.array([0, 0, 0])
    
    hit_object, hit_point, normal = None, None, None
    min_dist = float('inf')
    
    for obj in objects:
        hit = obj.intersect(ray)
        if hit and hit['distance'] < min_dist:
            min_dist = hit['distance']
            hit_object = obj
            hit_point = hit['point']
            normal = hit['normal']
    
    if hit_object is None:
        return np.array([0, 0, 0])  # Background color

    # Simple diffuse shading for demonstration
    color = np.zeros(3)
    for light in lights:
        to_light = light.position - hit_point
        to_light = to_light / np.linalg.norm(to_light)
        color += hit_object.material.color * light.intensity * max(np.dot(normal, to_light), 0)
    
    return color


def render(canvas, camera, objects, lights):
    for y in range(canvas.height):
        for x in range(canvas.width):
            u = (x + 0.5) / canvas.width
            v = (y + 0.5) / canvas.height
            ray = camera.get_ray(u, v)
            color = trace_ray(ray, objects, lights)
            canvas.set_pixel(x, y, color)

---

In [24]:
canvas = Canvas(100, 100)
camera = Camera([0, 10, 80], [0, 0, 0], [0, 1, 0], np.pi/4, canvas.width/canvas.height)
objects = load_obj("src/hand.obj")
lights = [Light([5, 5, 5], 2)]

render(canvas, camera, objects, lights)
canvas.save("output_opti.png")

---

In [4]:
from PIL import Image
from functools import reduce
import numpy as np
import time
import pywavefront
from concurrent.futures import ThreadPoolExecutor

class vec3():
    def __init__(self, x, y, z):
        (self.x, self.y, self.z) = (x, y, z)
    def __mul__(self, other):
        return vec3(self.x * other, self.y * other, self.z * other)
    def __add__(self, other):
        return vec3(self.x + other.x, self.y + other.y, self.z + other.z)
    def __sub__(self, other):
        return vec3(self.x - other.x, self.y - other.y, self.z - other.z)
    def dot(self, other):
        return (self.x * other.x) + (self.y * other.y) + (self.z * other.z)
    def cross(self, other):
        return vec3(self.y * other.z - self.z * other.y, self.z * other.x - self.x * other.z, self.x * other.y - self.y * other.x)
    def __abs__(self):
        return self.dot(self)
    def norm(self):
        mag = np.sqrt(abs(self))
        return self * (1.0 / np.where(mag == 0, 1, mag))
    def components(self):
        return (self.x, self.y, self.z)
rgb = vec3

(w, h) = (100, 100)         # Screen size
L = vec3(5, 5., -10)        # Point light position
E = vec3(0., 0.35, -1.)     # Eye position
FARAWAY = 1.0e39            # an implausibly huge distance

class Triangle:
    def __init__(self, v0, v1, v2, diffuse, mirror=0.5):
        self.v0 = v0
        self.v1 = v1
        self.v2 = v2
        self.diffuse = diffuse
        self.mirror = mirror
        self.normal = ((v1 - v0).cross(v2 - v0)).norm()
    
    def intersect(self, O, D):
        EPSILON = 1e-8
        edge1 = self.v1 - self.v0
        edge2 = self.v2 - self.v0
        h = D.cross(edge2)
        a = edge1.dot(h)
        if np.abs(a) < EPSILON:
            return FARAWAY
        f = 1.0 / a
        s = O - self.v0
        u = f * s.dot(h)
        if u < 0.0 or u > 1.0:
            return FARAWAY
        q = s.cross(edge1)
        v = f * D.dot(q)
        if v < 0.0 or u + v > 1.0:
            return FARAWAY
        t = f * edge2.dot(q)
        if t > EPSILON:
            return t
        return FARAWAY
    
    def diffusecolor(self, M):
        return self.diffuse
    
    def light(self, O, D, d, scene, bounce):
        M = (O + D * d)
        N = self.normal
        toL = (L - M).norm()
        toO = (E - M).norm()
        nudged = M + N * .0001

        light_distances = [s.intersect(nudged, toL) for s in scene]
        light_nearest = reduce(np.minimum, light_distances)
        seelight = light_distances[scene.index(self)] == light_nearest

        color = rgb(0.05, 0.05, 0.05)
        lv = np.maximum(N.dot(toL), 0)
        color += self.diffusecolor(M) * lv * seelight

        if bounce < 2:
            rayD = (D - N * 2 * D.dot(N)).norm()
            color += raytrace(nudged, rayD, scene, bounce + 1) * self.mirror

        phong = N.dot((toL + toO).norm())
        color += rgb(1, 1, 1) * np.power(np.clip(phong, 0, 1), 50) * seelight
        return color

def raytrace(O, D, scene, bounce = 0):
    distances = [s.intersect(O, D) for s in scene]
    nearest = reduce(np.minimum, distances, FARAWAY)
    color = rgb(0, 0, 0)
    for (s, d) in zip(scene, distances):
        if d != FARAWAY and d == nearest:
            color += s.light(O, D, d, scene, bounce)
    return color

# def load_obj(filename, diffuse=rgb(0.5, 0.5, 0.5)):
#     scene = []
#     mesh = pywavefront.Wavefront(filename, collect_faces=True)
#     vertices = mesh.vertices
#     for mesh in mesh.mesh_list:
#         for face in mesh.faces:
#             v0 = vec3(*vertices[face[0]])
#             v1 = vec3(*vertices[face[1]])
#             v2 = vec3(*vertices[face[2]])
#             scene.append(Triangle(v0, v1, v2, diffuse))
#     return scene

def load_obj(filename):
    diffuse = rgb(0.5, 0.5, 0.5)
    mesh = trimesh.load_mesh(filename)
    triangles = []
    for face in mesh.faces:
        v0, v1, v2 = [mesh.vertices[i] for i in face]
        triangles.append(Triangle(v0, v1, v2))
    return triangles

class Triangle:
    def __init__(self, v0, v1, v2):
        self.v0 = np.array(v0)
        self.v1 = np.array(v1)
        self.v2 = np.array(v2)
        self.normal = normalize(np.cross(v1 - v0, v2 - v0))

    def intersect(self, ray_origin, ray_dir):
        # Algorithme de Möller-Trumbore pour l'intersection de rayons avec un triangle
        epsilon = 1e-8
        edge1 = self.v1 - self.v0
        edge2 = self.v2 - self.v0
        h = np.cross(ray_dir, edge2)
        a = dot(edge1, h)
        if -epsilon < a < epsilon:
            return None
        f = 1.0 / a
        s = ray_origin - self.v0
        u = f * dot(s, h)
        if u < 0.0 or u > 1.0:
            return None
        q = np.cross(s, edge1)
        v = f * dot(ray_dir, q)
        if v < 0.0 or u + v > 1.0:
            return None
        t = f * dot(edge2, q)
        if t > epsilon:
            return t
        else:
            return None
        
def render_block(x_range, y_range, E, scene):
    block_width = int((x_range[1] - x_range[0]) * w)
    block_height = int((y_range[1] - y_range[0]) * h)
    x = np.linspace(x_range[0], x_range[1], block_width)
    y = np.linspace(y_range[0], y_range[1], block_height)
    X, Y = np.meshgrid(x, y)
    Q = vec3(X.ravel(), Y.ravel(), 0)
    D = (Q - E).norm()
    color_block = np.zeros((block_height, block_width, 3))
    for i, (x_val, y_val) in enumerate(zip(X.ravel(), Y.ravel())):
        color = raytrace(E, (vec3(x_val, y_val, 0) - E).norm(), scene)
        color_block[i // block_width, i % block_width] = color.components()
    return color_block

def render_image(scene, num_threads=12):
    x_splits = np.linspace(-1, 1, num_threads + 1)
    y_splits = np.linspace(-1 / (w / h) + .25, 1 / (w / h) + .25, num_threads + 1)

    blocks = [(x_splits[i], x_splits[i + 1], y_splits[j], y_splits[j + 1]) 
              for i in range(num_threads) 
              for j in range(num_threads)]

    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [executor.submit(render_block, (x_start, x_end), (y_start, y_end), E, scene) 
                   for (x_start, x_end, y_start, y_end) in blocks]
        results = [future.result() for future in futures]

    final_image = np.zeros((h, w, 3))
    for idx, (x_start, x_end, y_start, y_end) in enumerate(blocks):
        block_width = int((x_end - x_start) * w)
        block_height = int((y_end - y_start) * h)
        x_start_idx = int((x_start + 1) * w / 2)
        x_end_idx = int((x_end + 1) * w / 2)
        y_start_idx = int((y_start + 1 / (w / h) - 0.25) * h)
        y_end_idx = int((y_end + 1 / (w / h) - 0.25) * h)
        
        # Ensure indices are within bounds
        x_start_idx = max(0, x_start_idx)
        x_end_idx = min(w, x_end_idx)
        y_start_idx = max(0, y_start_idx)
        y_end_idx = min(h, y_end_idx)

        final_image[y_start_idx:y_end_idx, x_start_idx:x_end_idx] = results[idx]

    return final_image



# Load the scene and render
scene = load_obj("src/hand.obj")
start_time = time.time()
final_image = render_image(scene, num_threads=12)
print("Rendering took", time.time() - start_time)

# Save the image
Image.fromarray((255 * np.clip(final_image, 0, 1)).astype(np.uint8)).save("output_hand.png")


NameError: name 'normalize' is not defined